In [1]:
import pyaudio
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy.signal import butter, lfilter
import time
from tkinter import TclError

# to display in a separate Tk window
%matplotlib tk

# constants
CHUNK = 512               # reduced samples per frame to lighten load
FORMAT = pyaudio.paInt16   # audio format (16-bit int)
CHANNELS = 1               # single channel for microphone
RATE = 9600                # samples per second
UPDATE_INTERVAL = 5        # plot update interval (every 5 chunks)

# continuous plot configuration
NUM_CHUNKS = 20*10            # number of chunks to keep in the buffer
CHUNK_SIZE = CHUNK * NUM_CHUNKS  # total samples in buffer for plotting

# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# create a buffer to store the last 20 chunks
data_buffer = np.zeros(CHUNK_SIZE)

# x-axis for waveform and frequency spectrum
x = np.arange(0, 2 * CHUNK_SIZE, 2)      # x-axis for waveform (samples)
xf = np.linspace(0, RATE, CHUNK)         # x-axis for frequency spectrum

# plot lines for the waveform and spectrum
line, = ax1.plot(x, np.random.rand(CHUNK_SIZE), '-', lw=2)
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# Amplitude limit for display
AMPLITUDE_LIMIT = 4096*4

# format waveform plot
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(-AMPLITUDE_LIMIT, AMPLITUDE_LIMIT)
ax1.set_xlim(0, 2 * CHUNK_SIZE)
plt.setp(ax1, xticks=np.linspace(0, 2 * CHUNK_SIZE, 5), yticks=[-AMPLITUDE_LIMIT, 0, AMPLITUDE_LIMIT])

# format spectrum plot
ax2.set_xlim(20, RATE / 2)

# Design a bandpass filter from 15 to 500 Hz
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut=15.0, highcut=500.0, fs=RATE, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

# streaming loop
while True:
    # read audio data by chunks
    data = stream.read(CHUNK)
    data_np = np.frombuffer(data, dtype='h')  # convert to numpy array

    # Apply bandpass filter to the audio data
    filtered_data = bandpass_filter(data_np)

    # update the data buffer: roll and add new chunk at the end
    data_buffer = np.roll(data_buffer, -CHUNK)
    data_buffer[-CHUNK:] = filtered_data

    # Update the plot every few chunks to reduce load
    if frame_count % UPDATE_INTERVAL == 0:
        # update waveform plot with the buffer data
        line.set_ydata(data_buffer)

        # compute FFT of the current chunk and update frequency spectrum plot
        yf = fft(filtered_data)
        line_fft.set_ydata(np.abs(yf[0:CHUNK]) / (512 * CHUNK))

        # update the figure
        try:
            fig.canvas.draw()
            fig.canvas.flush_events()
        except TclError:
            # calculate average frame rate when stopping
            frame_rate = frame_count / (time.time() - start_time)
            print('stream stopped')
            print('average frame rate = {:.0f} FPS'.format(frame_rate))
            break

    frame_count += 1

# close the stream
stream.stop_stream()
stream.close()
p.terminate()


stream started


KeyboardInterrupt: 

In [2]:
import pyaudio
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
import time
from tkinter import TclError

# to display in a separate Tk window
%matplotlib tk

# constants
CHUNK = 512               # reduced samples per frame to lighten load
FORMAT = pyaudio.paInt16   # audio format (16-bit int)
CHANNELS = 1               # single channel for microphone
RATE = 9600                # samples per second
UPDATE_INTERVAL = 5        # plot update interval (every 5 chunks)

# continuous plot configuration
NUM_CHUNKS = 20*10            # number of chunks to keep in the buffer
CHUNK_SIZE = CHUNK * NUM_CHUNKS  # total samples in buffer for plotting

# create matplotlib figure and axis for waveform only
fig, ax1 = plt.subplots(figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# create a buffer to store the last 20 chunks
data_buffer = np.zeros(CHUNK_SIZE)

# x-axis for waveform
x = np.arange(0, 2 * CHUNK_SIZE, 2)      # x-axis for waveform (samples)

# plot line for the waveform
line, = ax1.plot(x, np.random.rand(CHUNK_SIZE), '-', lw=2)

# Amplitude limit for display
AMPLITUDE_LIMIT = 4096*4

# format waveform plot
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(-AMPLITUDE_LIMIT, AMPLITUDE_LIMIT)
ax1.set_xlim(0, 2 * CHUNK_SIZE)
plt.setp(ax1, xticks=np.linspace(0, 2 * CHUNK_SIZE, 5), yticks=[-AMPLITUDE_LIMIT, 0, AMPLITUDE_LIMIT])

# Design a bandpass filter from 15 to 500 Hz
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut=15.0, highcut=500.0, fs=RATE, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

# streaming loop
while True:
    # read audio data by chunks
    data = stream.read(CHUNK)
    data_np = np.frombuffer(data, dtype='h')  # convert to numpy array

    # Apply bandpass filter to the audio data
    filtered_data = bandpass_filter(data_np)

    # update the data buffer: roll and add new chunk at the end
    data_buffer = np.roll(data_buffer, -CHUNK)
    data_buffer[-CHUNK:] = filtered_data

    # Update the plot every few chunks to reduce load
    if frame_count % UPDATE_INTERVAL == 0:
        # update waveform plot with the buffer data
        line.set_ydata(data_buffer)

        # update the figure
        try:
            fig.canvas.draw()
            fig.canvas.flush_events()
        except TclError:
            # calculate average frame rate when stopping
            frame_rate = frame_count / (time.time() - start_time)
            print('stream stopped')
            print('average frame rate = {:.0f} FPS'.format(frame_rate))
            break

    frame_count += 1

# close the stream
stream.stop_stream()
stream.close()
p.terminate()


stream started


KeyboardInterrupt: 